# Task
We want to achieve the following workflow:

Step 1. 
1. Echo a string to msg.txt 
    
2. Echo a number and redirect it to results.txt 
    
Step 2.
1. Duplicate the content in msg.txt two times and redirect it to a new file
    
2. Get the value in results.txt and times the number by 2 and redirect it to results.txt

##### note: we used `ShellOPTemplate` to complete the task above. In this tutorial, we will achieve the same task using `PythonOPTemplate` 

import necessary packages

In [49]:
from pathlib import Path

from dflow import Step, Workflow, download_artifact, upload_artifact
from dflow.python import (OP, OPIO, Artifact, OPIOSign, PythonOPTemplate,
                          upload_packages)
from typing import List

if "__file__" in locals():
    upload_packages.append(__file__)

For step 1: 
This class can achieve the function to write files. In the example, we try to write one file containing message of string format, and one file including number of int format.
- input:
    - "msg"
    - "number"
- output:
    - two file.txt

In [50]:
class WriteFile(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'msg': str,
            'number': int,
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            "out_art": Artifact(List[Path])
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        f = open("msg.txt","w")
        f.write(op_in["msg"])
        f.close()
        g = open("results.txt","w")
        g.write(str(op_in["number"]))
        g.close()
        
        op_out = OPIO({
            "out_art": [Path("msg.txt"), Path("results.txt")],
        })

        return op_out

For Step 2

In [51]:
class Duplicate(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            "in_art": Artifact(Path),
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            "out_art": Artifact(List[Path])
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
    
        f = open(op_in["in_art"] / "msg.txt","r")
        content = f.read()
        f.close()
        f = open("bar.txt","w")
        f.write(content * 2)
        f.close()

        g = open(op_in["in_art"] / "results.txt","r")
        content = int(g.read())
        g.close()
        g = open("results.txt","w")
        g.write(str(content*2))
        g.close()
        
        op_out = OPIO({
            "out_art": [Path("bar.txt"), Path("results.txt")],
        })

        return op_out

In [52]:
step0 = Step(
    name="step0",
    template=PythonOPTemplate(WriteFile, image="python:3.8"),
    parameters={"msg": "HelloWorld!", "number": 1},
)

step1 = Step(
    name="step1",
    template=PythonOPTemplate(Duplicate, image="python:3.8"),
    artifacts={"in_art":step0.outputs.artifacts["out_art"]}
)
wf = Workflow(name="python")
wf.add(step0)
wf.add(step1)
wf.submit();

Workflow has been submitted (ID: python-fgkmz)
